# db2

In [2]:
from ipynb.fs.full.koselleck import *

In [3]:
PERIOD_LENS={5,20,70}

## Model definitions

In [4]:
#FN_DB=os.path.join(PATH_DATA,'db3.koselleck2.h5')
FN_DB='/home/ryan/db/db.koselleck6.shelve'
# os.remove(FN_DB)
DBOBJ=None

In [5]:
import shelve

In [23]:
# def get_db(mode='a'):
#     global DBOBJ
#     if DBOBJ is not None: return DBOBJ
#     DBOBJ=shelve.open(FN_DB)
#     return DBOBJ

In [8]:
db=get_db()
# print(db['/vecs/bpo/1720-1740/1'])

In [10]:
# db['/vecs/bpo/']

In [12]:
def get_db(): return shelve.open(FN_DB)
def db_has_key(key):
    with get_db() as db: return key in db
def dbget(key,default=None):
    with get_db() as db: return db.get(key,default)
def dbput(key,val):
    with get_db() as db: db[key]=val

In [13]:
# keys=list(db.keys())
# keys

In [14]:
db_has_key('test')

False

In [15]:
#with get_db() as db: db['test']=pd.Series(['Hello', 'world'])
db=get_db()

In [22]:
db['/dists/bpo/1720-1725']

,barrister,representation,november,peaceable,discharged,pared,collegiate,overwhelming,fiery,belles,...,ministerial,clearness,hospitable,twig,stretching,courting,annexation,calendar,mended,atom
abandon,1.063256,0.844460,1.104841,0.639787,0.994410,1.054900,1.014362,0.889311,0.846879,0.937167,...,0.791540,0.868854,0.992040,1.160458,0.730939,0.863703,0.918093,0.768445,1.022773,0.841005
abandoned,0.852291,0.836507,1.037614,0.935917,1.104277,1.093037,0.672470,0.938089,0.903250,0.730542,...,0.876004,0.785818,0.878354,0.941883,1.076768,0.974189,0.912069,0.692958,0.957376,0.965339
abandoning,0.878059,0.958804,1.153023,0.852423,1.166695,1.033038,0.952668,0.908733,0.930650,0.945803,...,0.946942,0.821491,0.649344,1.091615,0.943419,0.866298,1.010848,0.736502,1.021269,0.937601
abate,1.164719,1.065863,0.927512,1.102978,0.915663,0.956396,1.214211,0.822532,0.932770,1.114006,...,0.922670,0.794930,0.998595,0.803730,0.753091,0.997759,1.027717,0.958396,0.895621,0.995003
abated,1.044065,1.063268,0.970890,1.131560,0.912092,1.051303,1.005284,0.810237,0.963704,0.930457,...,1.106874,1.054008,0.932306,0.952673,0.842936,0.945340,1.207927,0.921007,0.895901,1.183953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youthful,0.874440,0.943297,1.156609,0.883709,1.146102,0.676462,0.895841,0.671855,0.558931,0.935306,...,0.669713,0.871665,0.888245,1.013186,0.545501,0.662950,0.734252,0.796994,1.157581,0.852324
zeal,1.150500,0.689286,1.290612,0.658810,1.233851,1.095378,1.011668,0.988736,1.013247,1.037740,...,1.005298,0.958817,0.830803,1.193517,0.938787,0.680164,0.811076,0.975247,0.780102,0.903489
zealous,0.876003,0.892029,1.218967,0.677643,1.125041,0.999065,0.918103,0.801278,0.978905,0.954751,...,0.959714,0.970654,0.581943,1.280254,1.048819,0.887393,0.880165,0.801695,0.991339,0.963490
zinc,1.113899,0.858197,0.726978,0.879184,1.070997,0.886705,0.903399,0.763011,0.896010,0.854380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# db['test']='hello'

## Vectors

In [13]:
dfmodels_all = get_pathdf_models(period_len=None)
# dfmodels_all

Scanning directory for models: 3901it [00:00, 65361.38it/s]


In [14]:
dfmodels=dfmodels_all[dfmodels_all.period_len.isin(PERIOD_LENS)]
# dfmodels

In [18]:
# %%timeit
# vwords=set(get_valid_words())

In [19]:
def do_ingest_vecs(row,only_valid_words=True):
    mpath=row.path
    m=load_model(mpath)
    data=m.wv.vectors
    keys=[m.wv.index_to_key[i] for i in range(len(data))]
    res=pd.DataFrame(data, index=keys)
    if only_valid_words:
        vwords=set(get_valid_words())
        res=res.loc[set(res.index) & vwords]
    qstr=f'/vecs/{row.corpus}/{row.period}/{int(row.run.split("_")[-1])}'
    return (qstr,res)

def ingest_vecs(dfmodels, only_valid_words=True,num_proc=1):
    objs = [row for i,row in dfmodels.iterrows()]
    iterr=pmap_iter(do_ingest_vecs, objs, num_proc=num_proc)
    db=get_db()
    for i,(qstr,qdf) in enumerate(iterr):
        db[qstr]=qdf
        if i and not i%10: db.sync()
    db.sync()

In [1]:
# ingest_vecs(dfmodels,num_proc=2)

In [26]:
vqstr='/vecs/bpo/1780-1800/1'

In [27]:
dbget(vqstr)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
barrister,0.150089,-0.940826,0.822163,-0.365061,0.491442,-0.411995,0.929706,2.074926,-1.925221,-1.305094,...,0.897786,-0.555819,1.645228,1.278834,0.311690,-0.262408,-1.797190,0.208491,-1.065541,-0.390771
representation,0.255103,-2.669126,0.106818,1.991442,-0.492201,-0.137021,0.191122,1.118572,0.735181,0.461479,...,1.542258,1.375813,-0.640469,0.351484,1.068309,-1.631528,1.397532,-1.509136,-2.584206,-3.159979
november,1.660902,-2.540474,-0.279772,1.163501,2.143204,-2.914960,1.219226,1.216411,-2.472668,-0.648752,...,1.107363,-0.668765,4.684813,-1.004731,0.252560,0.812193,0.870273,1.887735,2.356697,1.495078
peaceable,0.534629,2.119081,-2.962521,1.939062,-0.199213,0.741662,1.437396,-0.306059,1.297907,0.231939,...,-1.253074,1.305056,-0.823179,-1.752425,0.572528,-0.055916,1.273415,-0.574251,-1.107126,-2.122983
constituents,0.611275,1.647903,-1.113447,1.215214,-1.236166,-0.529411,1.404272,-0.698707,-3.168374,0.837553,...,-1.288349,0.566072,0.122897,-0.915087,1.003163,-0.440331,1.079429,-0.881100,-1.184188,-1.425080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
friendship,-2.186266,1.769814,2.303214,-3.220508,-1.432981,0.482882,1.440104,-2.866590,-1.367424,0.765299,...,-2.400832,-0.141235,-2.200494,-1.662177,0.932034,2.301400,-2.401198,1.459496,-0.456343,-0.368757
joyful,-0.005497,-1.214323,-1.383832,-0.369176,-1.520220,0.615671,-2.052162,-2.375713,-0.352698,1.075970,...,-1.021652,1.231817,-0.616074,-1.379638,-2.012111,-2.185333,1.947678,-0.464621,0.955209,-1.062166
deduction,-0.620607,-1.246100,-0.060162,1.209576,-0.644522,-0.659558,0.141621,1.472182,-1.058478,1.192855,...,0.143256,0.496181,0.925697,0.885267,0.823759,0.018422,0.307204,-0.239440,0.947469,-0.106131
enlargement,-1.708508,-0.234480,0.571475,0.435117,-0.604477,-0.098562,0.796185,-0.585601,-0.852158,0.538315,...,0.125303,0.756266,-1.054371,-0.257810,1.017378,-0.556893,-0.011494,-1.661778,0.969883,0.077509


In [28]:
def vecs(period,run=1,corpus=DEFAULT_CORPUS):
    qstr=f'/vecs/{corpus}/{period}/{run}'
    o=dbget(qstr)
    return pd.DataFrame() if o is None else o

In [29]:
vecs('1820-1840',332423)

""


## Distances

In [30]:
def vecs2dist(dfvecs,only_valid_words=True,lim=None):
    if only_valid_words:
        dfvecs=dfvecs.loc[
            set(dfvecs.index)&set(get_valid_words())
        ]
    if lim: dfvecs=dfvecs[:lim]
    dfsim=pd.DataFrame(
        fastdist.cosine_pairwise_distance(
            dfvecs.values.astype(float),
            return_matrix=True
        ),
        index=dfvecs.index,
        columns=dfvecs.index
    )
    dfdist=1-dfsim
    return dfdist

In [31]:
# X=vecs(random.choice(get_default_periods()))
# X

In [32]:
# o=vecs2dist(X)
# o

In [33]:
def dbgetdf(x):
    o=dbget(x)
    return o if o is not None else pd.DataFrame()

In [34]:
# dbgetdf('wrewere')

In [84]:
def do_ingest_dists(dfperiod,progress=False):
    #display(dfperiod)
    #return pd.DataFrame([{'a':len(dfperiod)}])
    df=None
    iterr=zip(dfperiod.corpus, dfperiod.period, dfperiod.run)
    if progress: iterr=tqdm(iterr,total=len(dfperiod))
    for corpus,period,run in iterr:
        run=int(run.split('_')[-1]) if type(run)==str else run
        if run>2: continue
        dfvecs=vecs(corpus=corpus, period=period, run=run)
        dfdist=vecs2dist(dfvecs)
        if df is None:
            df=dfdist
        else:
            df3=pd.concat([df,dfdist])
            df=df3.groupby(df3.index).mean()
#        print(df.shape)
    return df

def ingest_dists(dfmodels,num_proc=1,force=False):
    key_grp = [
        (f'/dists/{corpus}/{period}', gdf)
        for (corpus,period),gdf in dfmodels.groupby(['corpus','period'])
    ]
    if not force: key_grp=[(k,v) for k,v in key_grp if not db_has_key(k)]
    keys,grps=zip(*key_grp)
    iterr=pmap_iter(
        do_ingest_dists,
        grps,
        num_proc=num_proc
    )
    db=get_db()
    for key,res in zip(keys,iterr):
        db[key]=res
        db.sync()

In [85]:
for i,dfprd in dfmodels.groupby(['corpus','period']): pass

In [86]:
# do_ingest_dists(dfprd,progress=True)

In [ ]:
ingest_dists(dfmodels)

Mapping do_ingest_dists() [x1]:   2%|▏         | 1/46 [00:08<06:22,  8.50s/it]

In [21]:
# def ingest_dists_model(
#         period,run=1,corpus=DEFAULT_CORPUS,
#         only_valid_words=True,lim=25000,force=False,**attrs):
#     dfvecs=vecs(period,run)
#     dfdist=vecs2dist(dfvecs,only_valid_words=only_valid_words,lim=lim)
#     return dfdist

# def ingest_dists_model_(objd): return ingest_dists_model(**objd)

In [8]:
# def ingest_dists(dfmodels,only_valid_words=True,lim=None,num_proc=1,force=False):
#     dfmodels['run_int']=dfmodels.run.apply(lambda x: int(x.split('_')[-1]))
#     objs = [
#         dict(
#             period=period,
#             run=run,
#             corpus=corpus,
#             only_valid_words=only_valid_words,
#             lim=lim,
#             qstr=f'/dists/{corpus}/{period}/{run}'
#         )
#         for period,run,corpus in zip(dfmodels.period, dfmodels.run_int, dfmodels.corpus)
#     ]
#     if not force: objs = [d for d in objs if not db_has_key(d['qstr'])]
            
#     iterr=pmap_iter(ingest_dists_model_, objs[:100], num_proc=num_proc)
#     #with get_db(mode='a') as db:
#     db=get_db()
#     for i,(idx,odf) in enumerate(zip(objs, iterr)):
#         if odf is None: continue
#         db[idx['qstr']] = odf
#         #if i and not i%10: db.sync()
#         db.sync()

In [22]:
# ingest_dists(dfmodels,num_proc=1)

In [7]:
!ls -lSh ~/db/ | head

total 94G
-rw-rw-r-- 1 ryan ryan  50G Jun 24 10:39 db.koselleck5.shelve.dat
-rw-r--r-- 1 ryan ryan  13G Jun 20 16:58 db.koselleck.vecs.sqlite
-rw-r--r-- 1 ryan ryan 9.8G Jun  8 07:33 db.koselleck.nbr2.sqlite
-rw-r--r-- 1 ryan ryan 6.3G Jun 23 12:34 db.koselleck.distvecs_word.sqlite
-rw-r--r-- 1 ryan ryan 6.3G Jun 14 21:03 db.koselleck.nbr4.sqlite
-rw-r--r-- 1 ryan ryan 2.0G Jun 12 05:19 db.koselleck.psgs1.sqlite
-rw-r--r-- 1 ryan ryan 1.7G Jun 20 10:08 db.koselleck.distvecs_word2.sqlite
-rw-r--r-- 1 ryan ryan 1.2G Jun 11 17:49 db.koselleck.ttest.sqlite
-rw-r--r-- 1 ryan ryan 1.1G Jun  8 20:24 db.koselleck.lnm2.sqlite
